# Ноутбук для линейной модели

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.use_inf_as_na = True

import statsmodels.api as sm
import statsmodels.formula.api as smf

import scipy

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
%matplotlib notebook

from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('data/raifhack_train.csv', 
    parse_dates=['date'], 
    usecols=lambda x: x not in ['floor']
)

In [ ]:
# Есть 2 типа оценок: 
# 0 = из парсинга объявлений
# 1 = экспертные оценки
# Цены из объявлений содержат много шума, поэтому берем экспертные
# + Экспертных оценок меньше (около 5000), меньше времени на обучение
is_expert = (df['price_type'] == 1)
df = df[is_expert]

## Разделение на тренировочную и тестовую выборку

In [ ]:
split_by = df['date'].dt.month
month_count = split_by.value_counts(normalize=True)
month_count.sort_index().round(3) * 100

In [ ]:
for_test = df['date'].dt.month.isin([7, 8])
for_test.mean()

In [ ]:
df_train = df[ ~for_test ]
df_test = df[ for_test ]

## PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Отбираем только признаки вида "количество объектов типа х в радиусе y"
# В радиусе всегда есть точка (например: osm_catering_points_in_0.01)
pca_columns = df_train.columns[df_train.columns.str.contains('.', regex=False)]

In [ ]:
df_pca = df_train.select_dtypes(include=np.number)

df_pca = df_pca.loc[:, pca_columns]
# корреляция после логарифмирования больше
df_pca = df_pca.apply(np.log1p)

df_pca = (df_pca - df_pca.mean()) / df_pca.std()
df_pca = df_pca.fillna(df_pca.mean())

In [ ]:
pca = PCA()
df_pca = pca.fit_transform(df_pca)

In [ ]:
px.bar( pca.explained_variance_ratio_, template='plotly_white' )

## Признаки

In [ ]:
def get_features(df):
    df_features = pd.DataFrame()
    
#     no_transformation = [
#         # тут ничего не осталось
#     ]
#     for i in no_transformation:
#         df_features[i] = df[i]
    
    log_features = [
        'osm_city_closest_dist',
        'osm_crossing_closest_dist',
        'osm_subway_closest_dist',
        # признак оказался незначимым, убираем
        #'osm_train_stop_closest_dist',
        'osm_transport_stop_closest_dist',
        'reform_mean_floor_count_1000',
        'total_square',
        # Целевую переменную тоже логарифмируем
        'per_square_meter_price'
    ]
    for i in log_features:
        df_features[f'log_{i}'] = df[i].apply(np.log1p).values
    
    # добавляем первую главную компоненту
    df_features['pca_1'] = pca.transform(df[pca_columns]).T[0]
    df_features['pca_1'] /= 100
    
    # mean encoding для регионов
    region_means = df['per_square_meter_price'].apply(np.log).groupby(df['region']).mean().to_dict()
    df_features['region_means'] = df['region'].map(region_means).values
        
    return df_features

In [ ]:
df_train = df_train.pipe(get_features)
df_test = df_test.pipe(get_features)

In [ ]:
print(df_train.isna().sum().sum())
print(df_test.isna().sum().sum())

df_train = df_train.fillna(df_train.mean())
df_test = df_test.fillna(df_test.mean())

In [ ]:
# матрица корреляций признаков
px.imshow(df_train.drop(columns=['log_per_square_meter_price']).corr(), 
          color_continuous_scale=['red', 'white', 'blue'],
          color_continuous_midpoint=0)

In [ ]:
target_name = 'log_per_square_meter_price'

X_train = df_train.drop(columns=[target_name])
X_train = sm.add_constant(X_train)
y_train = df_train[target_name]

X_test = df_test.drop(columns=[target_name])
X_test = sm.add_constant(X_test)
y_test = df_test[target_name]

## Модель

In [ ]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary())

**Посмотрим на влияние первой главной компоненты**

In [ ]:
fig = px.scatter(
    x=df_train['pca_1'],
    y=df_train['log_per_square_meter_price'],
    trendline='ols',
    trendline_color_override='black',
    template='plotly_white'
)
fig.update_layout(xaxis_title='PC 1', yaxis_title='Log(Y)')
fig.data[0].marker.size=3
fig.show()

**Сравнение факта и модели (train)**

In [ ]:
fig = px.scatter(
    y=y_train,
    x=model.predict(X_train).values,
    trendline='ols',
    trendline_color_override='black',
    template='plotly_white'
)
fig.data[0].marker.size=3
fig.update_layout(xaxis_title='Модель', yaxis_title='Факт')
fig.show()

**Сравнение факта и модели (test)**

In [ ]:
fig = px.scatter(
    y=y_test,
    x=model.predict(X_test),
    trendline='ols',
    trendline_color_override='black',
    template='plotly_white'
)
fig.update_layout(xaxis_title='Модель', yaxis_title='Факт')
fig.data[0].marker.size=3
fig.show()

In [ ]:
px.histogram(
    model.predict(X_test),
    template='plotly_white'
)        

## Случайные остатки

In [ ]:
px.histogram( model.resid, template='plotly_white', nbins=80, marginal='box' ) 

In [ ]:
q = np.linspace(0, 1, 200)[1:-1]
fig = px.line(
    x=model.resid.quantile(q),
    y=model.resid.quantile(q),
    template='plotly_white'
)

fig.add_traces(px.scatter(
    x=model.resid.quantile(q),
    y=scipy.stats.norm(model.resid.mean(), model.resid.std()).ppf(q)
)['data'][0])

fig.data[1].marker.size=4
fig.data[0].line.color='lightgrey'
# fig.data[1].line.dash='dot'
fig.update_layout(xaxis_title='Факт', yaxis_title='Теория')
fig.show()

**Статистические тесты на нормальность**

In [ ]:
# Подвыборка для теста
# Хвосты обрезаем, иначе труднее делать тест Хи-квадрат
test_sample = model.resid.sample(250, random_state=0).clip(-1, 1.5)
# Теоретическое распределение - нормальное
theory_distr = scipy.stats.norm(model.resid.mean(), model.resid.std())
# Число бинов для теста Хи-квадрат
n_bins = np.ceil(5 * np.log10(model.resid.size))

In [ ]:
# Из предыдущей лабораторной работы
from lab1 import chi_square_bins

In [ ]:
f_obs, f_exp = chi_square_bins(
    test_sample, 
    theory_distr, 
    int(n_bins)
)

In [ ]:
scipy.stats.chisquare(f_obs, f_exp)

In [ ]:
scipy.stats.jarque_bera(test_sample)

## Регуляризация

In [ ]:
score_without_reg = r2_score(y_test, model.predict(X_test))
score_without_reg

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
# alpha_search = np.linspace(0.0001, 0.0030, 25)
alpha_search = np.linspace(0.0001, 40, 25)

reg_accuracy = [
    Ridge(alpha=alpha).fit(X_train, y_train).score(X_test, y_test)
    for alpha in alpha_search
]

In [ ]:
fig = px.line(
    x=alpha_search, 
    y=reg_accuracy, 
    template='plotly_white'
)

fig.add_traces(px.line(
    x=alpha_search, y=[score_without_reg]*len(alpha_search)
)['data'][0])

fig.data[0].mode = 'lines+markers'

fig.data[0].line.color = 'black'; fig.data[1].line.color = 'darkblue'
fig.update_layout(xaxis_title='alpha', yaxis_title='R^2 на тесте')

fig.show()

In [ ]:
# Прирост от регуляризации, в процентах
(max(reg_accuracy) - score_without_reg) * 100

## Доверительный интервал

In [ ]:
def get_error(model, x_new):
    # число наблюдений
    n = model.resid.size
    # число параметров
    # (применяется метод add_constant, 
    # поэтому +1 к числу параметров не нужно)
    k = x_new.size
    x = X_train.values
    # матричные операции
    inv = np.linalg.inv(np.dot(x.T, x))
    a = np.sqrt(np.dot(
        np.dot(x_new, inv),
        x_new
    ))
    # MSE
    mse = np.square(model.resid.values).mean()**0.5 
    mse *= np.sqrt( n/(n-k) )
    # распределение Стьюдента
    t = scipy.stats.t.ppf(1-0.05/2, n-2)
    
    return a * mse * abs(t)

In [ ]:
get_error(model, X_test.values[9])

In [ ]:
df_confidence_interval = pd.DataFrame()

df_confidence_interval['y_true'] = y_test
df_confidence_interval['y_pred'] = model.predict(X_test)
# считаем длину доверительного интервала
df_confidence_interval['error'] = [
    get_error(model, x_new)
    for x_new in X_test.values
]
# сверим со встроенным из statsmodels
df_confidence_interval['statsmodels_errors'] = (
    model
    .get_prediction(X_test)
    .summary_frame(alpha=0.05)
    .assign(statsmodels_errors=lambda x: 
            (x.mean_ci_upper-x.mean_ci_lower)/2)
)['statsmodels_errors']

In [ ]:
# сравним со встроенной функцией statsmodels
((
    df_confidence_interval['error'] - 
    df_confidence_interval['statsmodels_errors']
).abs() < 1e-7).all()

**Графики с доверительными интервалами**

In [ ]:
x_var_error_bar_plot = 'region_means'
fig = px.scatter(
    (
        df_confidence_interval
        .assign(x=X_test[x_var_error_bar_plot])
        # выберем точки равномерно по оси x
        .sort_values('x')
        .iloc[::10]
    ), 
    x='x', 
    y='y_pred',
    error_y='error',
    template='plotly_white'
)
fig.update_layout(xaxis_title=x_var_error_bar_plot)
fig.data[0].marker.size=1
fig.show()